# CAO 
***
## Author: Fionn McCarthy

## Introduction and Background
***

Below shows the libraries imported to carry out the relevant analysis

In [1]:
# Regular expressions 
import re

# Working with HTTP 
import requests as rq

# Data frames 
import pandas as pd

# Date and time library
import datetime as dt 

# For downloading from URl
import urllib.request as urlrq

import numpy as np

***
***
# 2021 CAO Data
***
### The link to the CAO points 2021 can be acccessed **[here](http://www.cao.ie/index.php?page=points&p=2021)**.
Over the course of the assignment the data for the 2021 Leaving Certificate points changed format on the website from HTML to Excel format. For teh purpose of this analysis I think it would be good idea to show how both formats can be extracted. In this case I will first show how regular expressions was utilized to extarct the 2021 CAo data in HTML form and then show how the Excel file was converted into daat for the purpose of this analysis.

In [2]:
# Retrieve the CAO points from the URl below
CAO1 = rq.get('http://www2.cao.ie/points/l8.php')

# View the above is okay, 200 means all okay. 
CAO1 

<Response [200]>

### Get the date and time of right now, in order to use for file creation time throughout notebook.
The datetime library is usesd in order to take a back up of the data retrieved from the CAO website.The reason the datetime library is used is to back up the data each time the notebook is run as to not overwrite the data. 

In [3]:
# date and time of right now.
now = dt.datetime.now()

# Formatting now as a string.
nowstring = now.strftime('%Y%m%d_%H%M%S')

### Saving the Original Data Set 

The server advises to utilize charset 'iso-8859-1' in order to decode the but from research this charset is unable to decode \x96 as it is not defined so we use 'cp1252' instead.

In [4]:
# Setting the charset to decode with 
CAO1.encoding = 'cp1252'

In [5]:
# Setting the charset to decode with 
CAO1.encoding = 'cp1252'

# Creating a file path for the data.
path2021_html = 'data/cao2021_' + nowstring + '.html'

# Saving the original html file before going any further. 
with open(path2021_html, 'w') as f:
    f.write(CAO1.text)

### Regular expressions selects the lines in the we want to keep.

In [6]:
# The regular expression is compiled below 
re_expr = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

In [7]:
# This function will split out the points that had either an '#' or '*' associated with them.
def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
    random = ''
    if s[-1] == '*':
        random = '*'
    points = ''
    for i in s:
        if i.isdigit():
            points = points + i
    # Return an array with points, portfolio, random        
    return [points, portfolio, random]

In [8]:
# Path creation for the csv file.
path2021_csv = 'data/cao2021_csv_' + nowstring + '.csv'

Looping through lines 

In [9]:
# Keep track of how many courses we process.
no_lines = 0

# Open and write to the cvs file
with open(path2021_csv, 'w') as f:
    # Write a header row for data set
    f.write(','.join(['code', 'title', 'points_r1', 'points_r2']) + '\n')
    # Loop through lines of the response
    for line in CAO1.iter_lines():
        # Decode using cp1252
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_expr.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code, first five characters in string
            course_code = dline[:5]
            # The course title
            course_title = dline[7:57].strip()
            # Course points split on one or more spaces after character 60
            course_points = re.split('  +', dline[60:]) 
            if len(course_points) != 2:
                course_points = course_points[:2]
            # join the fields using a comma 
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')
        
print(no_lines)

949


#### The above count of 949 courses was verified by a manual count that was carried out in visual studio code on original html file.

In [10]:
# Read csv file into 2021 dataframe
df2021 = pd.read_csv(path2021_csv, encoding='cp1252')

### 2021 Data change of Format on CAO website to excel, I will show how I converted this into data to be utlized in the analysis for this assignment.

In [11]:
# Creating a file path for the excel data.
path2021_excel = 'data/cao2021_' + nowstring + '.xlsx'

In [12]:
# https://stackoverflow.com/questions/19602931/basic-http-file-downloading-and-saving-to-disk-in-python
# Save original file
urlrq.urlretrieve("http://www2.cao.ie/points/CAOPointsCharts2021.xlsx", path2021_excel) 

('data/cao2021_20211212_210054.xlsx',
 <http.client.HTTPMessage at 0x1dfcd37dca0>)

In [13]:
# Download and parse the excel spreadsheet
# From examining the excel file I noted that needed to skip the first 10 trows of excel file that is loaded into dataframe
df2021_2 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2021.xlsx', skiprows = 11)

In [14]:
df2021_2

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
0,Engineering and engineering trades,Music and Instrument Technology,AL605,211,NaN,NaN,NaN,211,NaN,319,6,Athlone Institute of Technology,NaN,NaN,NaN
1,Health,Pharmacy Technician,AL630,308,NaN,NaN,NaN,308,NaN,409,6,Athlone Institute of Technology,NaN,NaN,NaN
2,Health,Dental Nursing,AL631,311,NaN,NaN,NaN,311,NaN,400,6,Athlone Institute of Technology,NaN,NaN,NaN
3,Biological and related sciences,Applied Science,AL632,297,NaN,NaN,NaN,297,NaN,454,6,Athlone Institute of Technology,NaN,NaN,NaN
4,Business and administration,Business,AL650,AQA,NaN,AQA,NaN,AQA,NaN,351,6,Athlone Institute of Technology,NaN,avp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,Information and Communication Technologies (ICTs),Creative Computing,WD211,270,NaN,NaN,NaN,270,NaN,392,8,Waterford Institute of Technology,NaN,NaN,NaN
1447,Personal services,Recreation and Sport Management,WD212,262,NaN,NaN,NaN,262,NaN,304,8,Waterford Institute of Technology,NaN,NaN,NaN
1448,Engineering and engineering trades,Mechanical and Manufacturing Engineering,WD230,230,NaN,230,NaN,230,NaN,361,8,Waterford Institute of Technology,NaN,avp,NaN
1449,Welfare,Early Childhood Care and Education,WD231,266,NaN,NaN,NaN,266,NaN,366,8,Waterford Institute of Technology,NaN,NaN,NaN


In [15]:
# Check of bottom value of 2021 file is correct
df2021_2.iloc[-1]

CATEGORY (ISCED Description)          Business and administration
Course Title                         Business Information Systems
Course Code                                                 WD232
R1 Points                                                     261
R1 Random                                                     NaN
R2 Points                                                     261
R2 Random                                                     NaN
EOS Points                                                    261
EOS Random                                                    NaN
EOS Midpoints                                                 329
Course Level                                                    8
HEI                             Waterford Institute of Technology
Test/Interview                                                NaN
AVP                                                           avp
v                                                             NaN
Name: 1450

In [16]:
df2021_2.dtypes

CATEGORY (ISCED Description)    object
Course Title                    object
Course Code                     object
R1 Points                       object
R1 Random                       object
R2 Points                       object
R2 Random                       object
EOS Points                      object
EOS Random                      object
EOS Midpoints                   object
Course Level                     int64
HEI                             object
Test/Interview                  object
AVP                             object
v                               object
dtype: object

In [17]:
df2021_2.columns

Index(['CATEGORY (ISCED Description)', 'Course Title', 'Course Code',
       'R1 Points', 'R1 Random', 'R2 Points ', 'R2 Random', 'EOS Points',
       'EOS Random', 'EOS Midpoints', 'Course Level', 'HEI', 'Test/Interview',
       'AVP', 'v'],
      dtype='object')

In [18]:
# https://stackoverflow.com/questions/44117326/how-can-i-remove-all-non-numeric-characters-from-all-the-values-in-a-particular
# Remove characters in columns that are non-numeric. 
# Note there is a space at end of Round 2 points col name
df2021_2['EOS Points'] = df2021_2['EOS Points'].replace('[A-Z.#*+a-z]', '', regex = True)
df2021_2['R1 Points'] = df2021_2['R1 Points'].replace('[A-Z.#*+a-z]', '', regex = True)
df2021_2['R2 Points '] = df2021_2['R2 Points '].replace('[A-Z.#*+a-z]', '', regex = True)
df2021_2['EOS Midpoints'] = df2021_2['EOS Midpoints'].replace('[A-Z.#*+a-z]', '', regex = True)

In [19]:
df2021_2

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
0,Engineering and engineering trades,Music and Instrument Technology,AL605,211,NaN,NaN,NaN,211,NaN,319,6,Athlone Institute of Technology,NaN,NaN,NaN
1,Health,Pharmacy Technician,AL630,308,NaN,NaN,NaN,308,NaN,409,6,Athlone Institute of Technology,NaN,NaN,NaN
2,Health,Dental Nursing,AL631,311,NaN,NaN,NaN,311,NaN,400,6,Athlone Institute of Technology,NaN,NaN,NaN
3,Biological and related sciences,Applied Science,AL632,297,NaN,NaN,NaN,297,NaN,454,6,Athlone Institute of Technology,NaN,NaN,NaN
4,Business and administration,Business,AL650,,NaN,,NaN,,NaN,351,6,Athlone Institute of Technology,NaN,avp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,Information and Communication Technologies (ICTs),Creative Computing,WD211,270,NaN,NaN,NaN,270,NaN,392,8,Waterford Institute of Technology,NaN,NaN,NaN
1447,Personal services,Recreation and Sport Management,WD212,262,NaN,NaN,NaN,262,NaN,304,8,Waterford Institute of Technology,NaN,NaN,NaN
1448,Engineering and engineering trades,Mechanical and Manufacturing Engineering,WD230,230,NaN,230,NaN,230,NaN,361,8,Waterford Institute of Technology,NaN,avp,NaN
1449,Welfare,Early Childhood Care and Education,WD231,266,NaN,NaN,NaN,266,NaN,366,8,Waterford Institute of Technology,NaN,NaN,NaN


In [20]:
# https://www.kite.com/python/answers/how-to-replace-each-empty-string-in-a-pandas-dataframe-with-nan-in-python
# Replace all blank strings with nulls in dataset
df2021_2 = df2021_2.replace(r'^\s*$', np.NaN, regex=True)

In [21]:
# Conver columns to floats
df2021_2['EOS Points'] = df2021_2['EOS Points'].astype(float)
df2021_2['R1 Points'] = df2021_2['R1 Points'].astype(float)
df2021_2['R2 Points '] = df2021_2['R2 Points '].astype(float)
df2021_2['EOS Midpoints'] = df2021_2['EOS Midpoints'].astype(float)


In [22]:
df2021_2.dtypes

CATEGORY (ISCED Description)     object
Course Title                     object
Course Code                      object
R1 Points                       float64
R1 Random                        object
R2 Points                       float64
R2 Random                        object
EOS Points                      float64
EOS Random                       object
EOS Midpoints                   float64
Course Level                      int64
HEI                              object
Test/Interview                   object
AVP                              object
v                                object
dtype: object

In [23]:
df2021_2

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v
0,Engineering and engineering trades,Music and Instrument Technology,AL605,211.0,NaN,NaN,NaN,211.0,NaN,319.0,6,Athlone Institute of Technology,NaN,NaN,NaN
1,Health,Pharmacy Technician,AL630,308.0,NaN,NaN,NaN,308.0,NaN,409.0,6,Athlone Institute of Technology,NaN,NaN,NaN
2,Health,Dental Nursing,AL631,311.0,NaN,NaN,NaN,311.0,NaN,400.0,6,Athlone Institute of Technology,NaN,NaN,NaN
3,Biological and related sciences,Applied Science,AL632,297.0,NaN,NaN,NaN,297.0,NaN,454.0,6,Athlone Institute of Technology,NaN,NaN,NaN
4,Business and administration,Business,AL650,NaN,NaN,NaN,NaN,NaN,NaN,351.0,6,Athlone Institute of Technology,NaN,avp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,Information and Communication Technologies (ICTs),Creative Computing,WD211,270.0,NaN,NaN,NaN,270.0,NaN,392.0,8,Waterford Institute of Technology,NaN,NaN,NaN
1447,Personal services,Recreation and Sport Management,WD212,262.0,NaN,NaN,NaN,262.0,NaN,304.0,8,Waterford Institute of Technology,NaN,NaN,NaN
1448,Engineering and engineering trades,Mechanical and Manufacturing Engineering,WD230,230.0,NaN,230.0,NaN,230.0,NaN,361.0,8,Waterford Institute of Technology,NaN,avp,NaN
1449,Welfare,Early Childhood Care and Education,WD231,266.0,NaN,NaN,NaN,266.0,NaN,366.0,8,Waterford Institute of Technology,NaN,NaN,NaN


In [24]:
# taking first 2 characters in Course Code column and outputting to new column hei_code column of df
df2021_2['hei_code'] = df2021_2['Course Code'].str[:2]
df2021_2

,CATEGORY (ISCED Description),Course Title,Course Code,R1 Points,R1 Random,R2 Points,R2 Random,EOS Points,EOS Random,EOS Midpoints,Course Level,HEI,Test/Interview,AVP,v,hei_code
0,Engineering and engineering trades,Music and Instrument Technology,AL605,211.0,NaN,NaN,NaN,211.0,NaN,319.0,6,Athlone Institute of Technology,NaN,NaN,NaN,AL
1,Health,Pharmacy Technician,AL630,308.0,NaN,NaN,NaN,308.0,NaN,409.0,6,Athlone Institute of Technology,NaN,NaN,NaN,AL
2,Health,Dental Nursing,AL631,311.0,NaN,NaN,NaN,311.0,NaN,400.0,6,Athlone Institute of Technology,NaN,NaN,NaN,AL
3,Biological and related sciences,Applied Science,AL632,297.0,NaN,NaN,NaN,297.0,NaN,454.0,6,Athlone Institute of Technology,NaN,NaN,NaN,AL
4,Business and administration,Business,AL650,NaN,NaN,NaN,NaN,NaN,NaN,351.0,6,Athlone Institute of Technology,NaN,avp,NaN,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,Information and Communication Technologies (ICTs),Creative Computing,WD211,270.0,NaN,NaN,NaN,270.0,NaN,392.0,8,Waterford Institute of Technology,NaN,NaN,NaN,WD
1447,Personal services,Recreation and Sport Management,WD212,262.0,NaN,NaN,NaN,262.0,NaN,304.0,8,Waterford Institute of Technology,NaN,NaN,NaN,WD
1448,Engineering and engineering trades,Mechanical and Manufacturing Engineering,WD230,230.0,NaN,230.0,NaN,230.0,NaN,361.0,8,Waterford Institute of Technology,NaN,avp,NaN,WD
1449,Welfare,Early Childhood Care and Education,WD231,266.0,NaN,NaN,NaN,266.0,NaN,366.0,8,Waterford Institute of Technology,NaN,NaN,NaN,WD


Validated, above is correct when compared to file.

#### Save Pandas Dataframe for 2021 Excel data  

In [25]:
# Creating a file path for the pandas data.
path2021_2_csv = 'data/cao2021_2_' + nowstring + '.csv'

In [26]:
# Save pandas dataframe to folder
df2021_2.to_csv(path2021_2_csv)

***
***
# 2020 CAO Data
***
### The link to the CAO points 2020 can be acccessed **[here](http://www.cao.ie/index.php?page=points&p=2020)**.

#### Save Original 2020 Data

In [27]:
# Creating a file path for the excel data.
path2020_excel = 'data/cao2020_' + nowstring + '.xlsx'

In [28]:
# https://stackoverflow.com/questions/19602931/basic-http-file-downloading-and-saving-to-disk-in-python
# Save original file
urlrq.urlretrieve("http://www2.cao.ie/points/CAOPointsCharts2020.xlsx", path2020_excel) 

('data/cao2020_20211212_210054.xlsx',
 <http.client.HTTPMessage at 0x1dfcde80400>)

#### Load Spreadhseet using Pandas

In [29]:
# Download and parse the excel spreadsheet
# From examining the excel file I noted that needed to skip the first 10 trows of excel file that is loaded into dataframe
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows = 10)

In [30]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [32]:
df2020.dtypes

CATEGORY (i.e.ISCED description)     object
COURSE TITLE                         object
COURSE CODE2                         object
R1 POINTS                            object
R1 Random *                          object
R2 POINTS                            object
R2 Random*                           object
EOS                                  object
EOS Random *                         object
EOS Mid-point                        object
LEVEL                                 int64
HEI                                  object
Test/Interview #                     object
avp                                  object
v                                    object
Column1                             float64
Column2                             float64
Column3                             float64
Column4                             float64
Column5                             float64
Column6                             float64
Column7                             float64
Column8                         

In [33]:
# https://stackoverflow.com/questions/44117326/how-can-i-remove-all-non-numeric-characters-from-all-the-values-in-a-particular
# Remove characters in columns that are non-numeric. 
# Note there is a space at end of Round 2 points col name
df2020['EOS'] = df2020['EOS'].replace('[A-Z.#*+a-z]', '', regex = True)
df2020['R1 POINTS'] = df2020['R1 POINTS'].replace('[A-Z.#*+a-z]', '', regex = True)
df2020['R2 POINTS'] = df2020['R2 POINTS'].replace('[A-Z.#*+a-z]', '', regex = True)
df2020['EOS Mid-point'] = df2020['EOS Mid-point'].replace('[A-Z.#*+a-z]', '', regex = True)

# https://www.kite.com/python/answers/how-to-replace-each-empty-string-in-a-pandas-dataframe-with-nan-in-python
# Replace all blank strings with nulls in dataset
df2020 = df2020.replace(r'^\s*$', np.NaN, regex = True)

# Conver EOS column to float 
df2020['EOS'] = df2020['EOS'].astype(float)
df2020['R1 POINTS'] = df2020['R1 POINTS'].astype(float)
df2020['R2 POINTS'] = df2020['R2 POINTS'].astype(float)
df2020['EOS Mid-point'] = df2020['EOS Mid-point'].astype(float)

In [34]:
df2020.dtypes

CATEGORY (i.e.ISCED description)     object
COURSE TITLE                         object
COURSE CODE2                         object
R1 POINTS                           float64
R1 Random *                          object
R2 POINTS                           float64
R2 Random*                           object
EOS                                 float64
EOS Random *                         object
EOS Mid-point                       float64
LEVEL                                 int64
HEI                                  object
Test/Interview #                     object
avp                                  object
v                                    object
Column1                             float64
Column2                             float64
Column3                             float64
Column4                             float64
Column5                             float64
Column6                             float64
Column7                             float64
Column8                         

In [35]:
# taking first 2 characters in Course Code column and outputting to new column hei_code column of df
df2020['hei_code'] = df2020['COURSE CODE2'].str[:2]
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,hei_code
0,Business and administration,International Business,AC120,209.0,NaN,NaN,NaN,209.0,NaN,280.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AC
1,Humanities (except languages),Liberal Arts,AC137,252.0,NaN,NaN,NaN,252.0,NaN,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AC
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AD
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AD
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188.0,NaN,NaN,NaN,188.0,NaN,339.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279.0,NaN,NaN,NaN,279.0,NaN,337.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271.0,NaN,NaN,NaN,271.0,NaN,318.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD
1462,Personal services,Recreation and Sport Management,WD212,270.0,NaN,NaN,NaN,270.0,NaN,349.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WD


#### Save Pandas Dataframe

In [36]:
# Creating a file path for the pandas data.
path2020_csv = 'data/cao2020_' + nowstring + '.csv'

In [37]:
# Save pandas datafram to folder
df2020.to_csv(path2020_csv)

***
***
# 2019 CAO Data
***
### The link to the CAO points 2019 can be acccessed **[here](http://www.cao.ie/index.php?page=points&p=2019)**.
The level 8 data and the level 6 and 7 data are held on two different files so I will first manipulate the level 8 data and then the level 6 and 7 data. 
### Level 8 data


#### In order to reproduce the data these are the steps I followed 
1. Download the original level 8 PDF file from the CAO website (link above). 
2. Open the original file using Microsoft Word. 
3. When open in Word, save the document as a word document. 
4. Re-save the Word document for editing.
5. Delete headers and page numbers in footer.
6. Select all in document and copy. 
7. Paste into Notepad++.
8. Remove institution name lines and blank lines.
9. In column headings, replace 'COURSE AND INSTITUTION' with 'Course', save file into folder as a csv file.
10. Change backticks to apostrophes. 
11. Delete tabs at end of lines: 61, 64, 169, 171, 172, 200, 335, 352, 425, 433, 700, 701, 752, 793, 830, 837, 869 and 894. 
12. Remove tab after 'Mid' first line.
13. Remove double tab on line 37.
14. Delete tabs at end of lines: 28, 107, 178, 201, 266, 370, 373, 391, 415, 437, 464, 494, 518, 535, 557, 571, 604, 625, 670, 679, 689, 785, 795
15. This data was then validated by pasting original data into visual studio code and getting the same number of rows.

In [38]:
df2019 = pd.read_csv('data/cao2019_20211130_205230_edited.csv', sep='\t')

In [39]:
df2019

,Course Code,Course,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,AL802,Software Design with Cloud Computing,301,306.0
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,AL805,Network Management and Cloud Infrastructure,329,442.0
4,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
925,WD200,Arts (options),221,296.0
926,WD210,Software Systems Development,271,329.0
927,WD211,Creative Computing,275,322.0
928,WD212,Recreation and Sport Management,274,311.0


In [40]:
# Adding a column to the dataframe specifying the course level, here level 8
df2019['Level'] = 8

In [41]:
df2019

,Course Code,Course,EOS,Mid,Level
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0,8
1,AL802,Software Design with Cloud Computing,301,306.0,8
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0,8
3,AL805,Network Management and Cloud Infrastructure,329,442.0,8
4,AL810,Quantity Surveying,307,349.0,8
...,...,...,...,...,...
925,WD200,Arts (options),221,296.0,8
926,WD210,Software Systems Development,271,329.0,8
927,WD211,Creative Computing,275,322.0,8
928,WD212,Recreation and Sport Management,274,311.0,8


### Level 6 & 7 data

#### In order to reproduce the data these are the steps I followed 
1. Download the original level 6 & 7 PDF file from the CAO website (link above). 
2. Open the original file using Microsoft Word. 
3. When open in Word, save the document as a word document. 
4. Re-save the Word document for editing.
5. Delete headers and page numbers in footer.
6. Select all in document and copy. 
7. Paste into Notepad++.
8. Remove institution name lines and blank lines.
9. In column headings, replace 'COURSE AND INSTITUTION' with 'Course', save file into folder as a csv file.
10. Remove tab after 'Mid' first line.
11. Delete tabs at end of lines: 31, 67, 69, 104, 108, 124, 157, 158, 174, 220, 224, 225, 228, 249, 294, 295, 328, 331, 365, 366, 410 and 433. 
12. This data was then validated by pasting original data into visual studio code and getting the same number of rows.

In [42]:
df2019_67 = pd.read_csv('data/cao2019_67_20211203_222130_edited.csv', sep='\t')

In [43]:
df2019_67

,Course Code,Course,EOS,Mid
0,AL600,Software Design,205,306.0
1,AL601,Computer Engineering,196,272.0
2,AL602,Mechanical Engineering,258,424.0
3,AL604,Civil Engineering,252,360.0
4,AL630,Pharmacy Technician,306,366.0
...,...,...,...,...
456,WD188,Applied Health Care,206,339.0
457,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0
458,WD206,Electronic Engineering,191,322.0
459,WD207,Mechanical Engineering,179,330.0


In [44]:
# Adding a column to the dataframe specifying the course level, here level 6 and 7
df2019_67['Level'] = '6 or 7'

In [45]:
df2019_67

,Course Code,Course,EOS,Mid,Level
0,AL600,Software Design,205,306.0,6 or 7
1,AL601,Computer Engineering,196,272.0,6 or 7
2,AL602,Mechanical Engineering,258,424.0,6 or 7
3,AL604,Civil Engineering,252,360.0,6 or 7
4,AL630,Pharmacy Technician,306,366.0,6 or 7
...,...,...,...,...,...
456,WD188,Applied Health Care,206,339.0,6 or 7
457,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0,6 or 7
458,WD206,Electronic Engineering,191,322.0,6 or 7
459,WD207,Mechanical Engineering,179,330.0,6 or 7


### Combining both 2019 dataframes into one containing level 6, level 7 and level 8 data.

In [46]:
# Combine both dataframes
df2019 = pd.concat([df2019, df2019_67], ignore_index=True)
df2019

,Course Code,Course,EOS,Mid,Level
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0,8
1,AL802,Software Design with Cloud Computing,301,306.0,8
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0,8
3,AL805,Network Management and Cloud Infrastructure,329,442.0,8
4,AL810,Quantity Surveying,307,349.0,8
...,...,...,...,...,...
1386,WD188,Applied Health Care,206,339.0,6 or 7
1387,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0,6 or 7
1388,WD206,Electronic Engineering,191,322.0,6 or 7
1389,WD207,Mechanical Engineering,179,330.0,6 or 7


Manipulate 2019 data to take out # and * from column and create new columns. Just leave numbers in points col.

In [47]:
# Find a record with a '#' in EOS column of data set, use this to test code below. 
df2019.loc[df2019['Course Code']=='AL763']

,Course Code,Course,EOS,Mid,Level
957,AL763,Graphic Design (portfolio),#626,820.0,6 or 7


In [48]:
# Find a record with a '*' in EOS column of data set, use this to test code below. 
df2019.loc[df2019['Course Code']=='CK201']

,Course Code,Course,EOS,Mid,Level
124,CK201,Commerce,465*,489.0,8


In [49]:
# https://stackoverflow.com/questions/44117326/how-can-i-remove-all-non-numeric-characters-from-all-the-values-in-a-particular
# Remove characters in EOS column that are not numeric. 
df2019['EOS'] = df2019['EOS'].str.extract('(\d+)', expand=False)

In [50]:
# Validate if this worked
df2019.loc[df2019['Course Code']=='CK201']

,Course Code,Course,EOS,Mid,Level
124,CK201,Commerce,465,489.0,8


In [51]:
# Validate if this worked
df2019.loc[df2019['Course Code']=='AL763']

,Course Code,Course,EOS,Mid,Level
957,AL763,Graphic Design (portfolio),626,820.0,6 or 7


In [52]:
df2019

,Course Code,Course,EOS,Mid,Level
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0,8
1,AL802,Software Design with Cloud Computing,301,306.0,8
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0,8
3,AL805,Network Management and Cloud Infrastructure,329,442.0,8
4,AL810,Quantity Surveying,307,349.0,8
...,...,...,...,...,...
1386,WD188,Applied Health Care,206,339.0,6 or 7
1387,WD205,Molecular Biology with Biopharmaceutical Science,208,441.0,6 or 7
1388,WD206,Electronic Engineering,191,322.0,6 or 7
1389,WD207,Mechanical Engineering,179,330.0,6 or 7


In [53]:
df2019.dtypes

Course Code     object
Course          object
EOS             object
Mid            float64
Level           object
dtype: object

In [54]:
# Conver EOS column to float 
df2019['EOS'] = df2019['EOS'].astype(float)

In [55]:
df2019.dtypes

Course Code     object
Course          object
EOS            float64
Mid            float64
Level           object
dtype: object

In [56]:
df2019

,Course Code,Course,EOS,Mid,Level
0,AL801,Software Design with Virtual Reality and Gaming,304.0,328.0,8
1,AL802,Software Design with Cloud Computing,301.0,306.0,8
2,AL803,Software Design with Mobile Apps and Connected...,309.0,337.0,8
3,AL805,Network Management and Cloud Infrastructure,329.0,442.0,8
4,AL810,Quantity Surveying,307.0,349.0,8
...,...,...,...,...,...
1386,WD188,Applied Health Care,206.0,339.0,6 or 7
1387,WD205,Molecular Biology with Biopharmaceutical Science,208.0,441.0,6 or 7
1388,WD206,Electronic Engineering,191.0,322.0,6 or 7
1389,WD207,Mechanical Engineering,179.0,330.0,6 or 7


The 2019 data does not contain the institution name so I will use the course code and 2020 data in order to retrieve these higher education institute (HEI) names. 

In [57]:
# taking first 2 characters in Course Code column and outputting to new column hei_code column of df
df2019['hei_code'] = df2019['Course Code'].str[:2]
df2019

,Course Code,Course,EOS,Mid,Level,hei_code
0,AL801,Software Design with Virtual Reality and Gaming,304.0,328.0,8,AL
1,AL802,Software Design with Cloud Computing,301.0,306.0,8,AL
2,AL803,Software Design with Mobile Apps and Connected...,309.0,337.0,8,AL
3,AL805,Network Management and Cloud Infrastructure,329.0,442.0,8,AL
4,AL810,Quantity Surveying,307.0,349.0,8,AL
...,...,...,...,...,...,...
1386,WD188,Applied Health Care,206.0,339.0,6 or 7,WD
1387,WD205,Molecular Biology with Biopharmaceutical Science,208.0,441.0,6 or 7,WD
1388,WD206,Electronic Engineering,191.0,322.0,6 or 7,WD
1389,WD207,Mechanical Engineering,179.0,330.0,6 or 7,WD


#### Save Pandas Dataframe

In [58]:
# Creating a file path for the pandas data.
path2019_csv = 'data/cao2019_' + nowstring + '.csv'

# Save pandas datafram to folder
df2019.to_csv(path2019_csv)

***
***
# Joining the three years of data together.
***
## Concat join

In [59]:
# Create df for course in each year
courses2021 = df2021_2[['Course Code', 'Course Title', 'HEI', 'hei_code']]
courses2021.columns = ['code', 'title', 'hei', 'hei_code']

courses2020 = df2020[['COURSE CODE2', 'COURSE TITLE', 'HEI', 'hei_code']]
courses2020.columns = ['code', 'title', 'hei', 'hei_code']

courses2019 = df2019[['Course Code', 'Course', 'hei_code']]
courses2019.columns = ['code', 'title', 'hei_code']
courses2019

,code,title,hei_code
0,AL801,Software Design with Virtual Reality and Gaming,AL
1,AL802,Software Design with Cloud Computing,AL
2,AL803,Software Design with Mobile Apps and Connected...,AL
3,AL805,Network Management and Cloud Infrastructure,AL
4,AL810,Quantity Surveying,AL
...,...,...,...
1386,WD188,Applied Health Care,WD
1387,WD205,Molecular Biology with Biopharmaceutical Science,WD
1388,WD206,Electronic Engineering,WD
1389,WD207,Mechanical Engineering,WD


In [60]:
# Combine all three years of courses
allcourses = pd.concat([courses2021, courses2020, courses2019], ignore_index=True)
allcourses

,code,title,hei,hei_code
0,AL605,Music and Instrument Technology,Athlone Institute of Technology,AL
1,AL630,Pharmacy Technician,Athlone Institute of Technology,AL
2,AL631,Dental Nursing,Athlone Institute of Technology,AL
3,AL632,Applied Science,Athlone Institute of Technology,AL
4,AL650,Business,Athlone Institute of Technology,AL
...,...,...,...,...
4301,WD188,Applied Health Care,NaN,WD
4302,WD205,Molecular Biology with Biopharmaceutical Science,NaN,WD
4303,WD206,Electronic Engineering,NaN,WD
4304,WD207,Mechanical Engineering,NaN,WD


In [61]:
# Sort values by code 
allcourses.sort_values('code')

,code,title,hei,hei_code
273,AC120,International Business,American College,AC
1451,AC120,International Business,American College,AC
3083,AC120,International Business,NaN,AC
1452,AC137,Liberal Arts,American College,AC
3084,AC137,Liberal Arts,NaN,AC
...,...,...,...,...
3844,WD230,Mechanical and Manufacturing Engineering,NaN,WD
2914,WD230,Mechanical and Manufacturing Engineering,Waterford Institute of Technology,WD
1448,WD230,Mechanical and Manufacturing Engineering,Waterford Institute of Technology,WD
1449,WD231,Early Childhood Care and Education,Waterford Institute of Technology,WD


In [62]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated()]

,code,title,hei,hei_code
1451,AC120,International Business,American College,AC
1452,AC137,Liberal Arts,American College,AC
1454,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD
1457,AD204,Fine Art (portfolio),National College of Art and Design,AD
1458,AD211,Fashion Design (portfolio),National College of Art and Design,AD
...,...,...,...,...
2910,WD208,Manufacturing Engineering,Waterford Institute of Technology,WD
2911,WD210,Software Systems Development,Waterford Institute of Technology,WD
2912,WD211,Creative Computing,Waterford Institute of Technology,WD
2913,WD212,Recreation and Sport Management,Waterford Institute of Technology,WD


In [63]:
# Returns a copy of the data frame with duplciates removed.
allcourses.drop_duplicates()

,code,title,hei,hei_code
0,AL605,Music and Instrument Technology,Athlone Institute of Technology,AL
1,AL630,Pharmacy Technician,Athlone Institute of Technology,AL
2,AL631,Dental Nursing,Athlone Institute of Technology,AL
3,AL632,Applied Science,Athlone Institute of Technology,AL
4,AL650,Business,Athlone Institute of Technology,AL
...,...,...,...,...
4301,WD188,Applied Health Care,NaN,WD
4302,WD205,Molecular Biology with Biopharmaceutical Science,NaN,WD
4303,WD206,Electronic Engineering,NaN,WD
4304,WD207,Mechanical Engineering,NaN,WD


In [64]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated(subset=['code'])]

,code,title,hei,hei_code
1451,AC120,International Business,American College,AC
1452,AC137,Liberal Arts,American College,AC
1453,AD101,"First Year Art & Design (Common Entry,portfolio)",National College of Art and Design,AD
1454,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD
1455,AD103,Textile & Surface Design and Jewellery & Objec...,National College of Art and Design,AD
...,...,...,...,...
4301,WD188,Applied Health Care,NaN,WD
4302,WD205,Molecular Biology with Biopharmaceutical Science,NaN,WD
4303,WD206,Electronic Engineering,NaN,WD
4304,WD207,Mechanical Engineering,NaN,WD


In [65]:
# Returns a copy of the data frame with duplciates removed - based only on code.
# We must ignore index in order to not bring through index
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)
allcourses

,code,title,hei,hei_code
0,AL605,Music and Instrument Technology,Athlone Institute of Technology,AL
1,AL630,Pharmacy Technician,Athlone Institute of Technology,AL
2,AL631,Dental Nursing,Athlone Institute of Technology,AL
3,AL632,Applied Science,Athlone Institute of Technology,AL
4,AL650,Business,Athlone Institute of Technology,AL
...,...,...,...,...
1807,SG432,Environmental Protection,NaN,SG
1808,SG438,Applied Archaeology,NaN,SG
1809,TL712,Computing with Digital Media,NaN,TL
1810,TL719,Culinary Arts,NaN,TL


I will utilize the same logic as above in order to populated the null values for the HEI that were not brought over from 2019 dataframe. I will use the 2020 and 2021 data in order to retrive the HEI and join it on below. 

In [66]:
#Create an array just with courses
colleges2021 = df2021_2[['HEI', 'hei_code']]
colleges2021.columns = ['hei', 'hei_code']

colleges2020 = df2020[['HEI', 'hei_code']]
colleges2020.columns = ['hei', 'hei_code']

colleges2019 = df2019[['hei_code']]


# Combine all three years of courses
allcolleges = pd.concat([colleges2021, colleges2020, colleges2020], ignore_index=True)
allcolleges

# Sort values by code 
allcolleges.sort_values('hei_code')

# Finds all extra copies of duplicated rows.
allcolleges[allcolleges.duplicated()]

# Returns a copy of the data frame with duplciates removed.
allcolleges.drop_duplicates()

# Finds all extra copies of duplicated rows.
allcolleges[allcolleges.duplicated(subset=['hei_code'])]

# Returns a copy of the data frame with duplciates removed - based only on code.
# We must ignore index in order to not bring through index
allcolleges.drop_duplicates(subset=['hei_code'], inplace=True, ignore_index=True)

allcolleges.columns = ['hei2', 'hei_code']

allcolleges

,hei2,hei_code
0,Athlone Institute of Technology,AL
1,"Institute of Technology, Carlow",CW
2,"Carlow College, St. Patrick`s",PC
3,Cork Institute of Technology,CR
4,University College Cork (NUI),CK
5,American College,AC
6,CCT College Dublin,CT
7,Marino Institute of Education,CM
8,Dorset College,DS
9,Dublin Business School,DB


In [67]:
# Set the index to the code column.
allcolleges.set_index('hei_code', inplace=True)
allcolleges

,hei2
hei_code,
AL,Athlone Institute of Technology
CW,"Institute of Technology, Carlow"
PC,"Carlow College, St. Patrick`s"
CR,Cork Institute of Technology
CK,University College Cork (NUI)
AC,American College
CT,CCT College Dublin
CM,Marino Institute of Education
DS,Dorset College


In [68]:
# Set the index to the code column.
allcourses.set_index('hei_code', inplace=True)
allcourses

,code,title,hei
hei_code,,,
AL,AL605,Music and Instrument Technology,Athlone Institute of Technology
AL,AL630,Pharmacy Technician,Athlone Institute of Technology
AL,AL631,Dental Nursing,Athlone Institute of Technology
AL,AL632,Applied Science,Athlone Institute of Technology
AL,AL650,Business,Athlone Institute of Technology
...,...,...,...
SG,SG432,Environmental Protection,NaN
SG,SG438,Applied Archaeology,NaN
TL,TL712,Computing with Digital Media,NaN


In [69]:
allcourses = allcourses.join(allcolleges)
allcourses

,code,title,hei,hei2
hei_code,,,,
AC,AC120,International Business,American College,American College
AC,AC137,Liberal Arts,American College,American College
AD,AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,National College of Art and Design
AD,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,National College of Art and Design
AD,AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,National College of Art and Design
...,...,...,...,...
WD,WD231,Early Childhood Care and Education,Waterford Institute of Technology,Waterford Institute of Technology
WD,WD232,Business Information Systems,Waterford Institute of Technology,Waterford Institute of Technology
WD,WD139,Civil Engineering,Waterford Institute of Technology,Waterford Institute of Technology


In [70]:
# https://stackoverflow.com/questions/40245507/python-pandas-selecting-rows-whose-column-value-is-null-none-nan
null_hei = allcourses[allcourses['hei2'].isna()]

In [71]:
null_hei

,code,title,hei,hei2
hei_code,,,,
BN,BN101,Business,NaN,NaN
BN,BN103,Business and Information Technology,NaN,NaN
BN,BN104,Computing (Information Technology),NaN,NaN
BN,BN107,Applied Social Studies in Social Care,NaN,NaN
BN,BN108,Engineering (Common Entry with Award options),NaN,NaN
...,...,...,...,...
TA,TA312,Computing,NaN,NaN
TA,TA313,Information Technology Management,NaN,NaN
TA,TA314,Pharmaceutical Science,NaN,NaN


In [72]:
# https://www.machinelearningplus.com/pandas/pandas-reset-index/
null_hei = null_hei.reset_index()
null_hei

,hei_code,code,title,hei,hei2
0,BN,BN101,Business,NaN,NaN
1,BN,BN103,Business and Information Technology,NaN,NaN
2,BN,BN104,Computing (Information Technology),NaN,NaN
3,BN,BN107,Applied Social Studies in Social Care,NaN,NaN
4,BN,BN108,Engineering (Common Entry with Award options),NaN,NaN
...,...,...,...,...,...
185,TA,TA312,Computing,NaN,NaN
186,TA,TA313,Information Technology Management,NaN,NaN
187,TA,TA314,Pharmaceutical Science,NaN,NaN
188,TA,TA315,DNA and Forensic Analysis,NaN,NaN


In [73]:
# Count distinct hei codes in null_hei
print(null_hei['hei_code'].unique())

['BN' 'DT' 'TA']


From review of the 2019 dataset I found BA to be 'Technological University Dublin Blanchardstown Campus', DT to be 'Technological University Dublin City Campus' and TA to be 'Technological University Dublin Tallaght Campus'. I will 

In [74]:
allcourses = allcourses.reset_index()
allcourses

,hei_code,code,title,hei,hei2
0,AC,AC120,International Business,American College,American College
1,AC,AC137,Liberal Arts,American College,American College
2,AD,AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,National College of Art and Design
3,AD,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,National College of Art and Design
4,AD,AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,National College of Art and Design
...,...,...,...,...,...
1807,WD,WD231,Early Childhood Care and Education,Waterford Institute of Technology,Waterford Institute of Technology
1808,WD,WD232,Business Information Systems,Waterford Institute of Technology,Waterford Institute of Technology
1809,WD,WD139,Civil Engineering,Waterford Institute of Technology,Waterford Institute of Technology
1810,WD,WD149,Early Childhood Studies,Waterford Institute of Technology,Waterford Institute of Technology


In [75]:
allcourses.loc[allcourses.hei_code == 'BN', ['hei2']] = 'Technological University Dublin Blanchardstown Campus'
allcourses.loc[allcourses.hei_code == 'DT', ['hei2']] = 'Technological University Dublin City Campus'
allcourses.loc[allcourses.hei_code == 'TA', ['hei2']] = 'Technological University Dublin Tallaght Campus'

In [76]:
null_hei2 = allcourses[allcourses['hei2'].isna()]
null_hei2

,hei_code,code,title,hei,hei2


In [77]:
# Count of null values in columns as sanity check
hei_null = allcourses['hei2'].isnull().sum(axis = 0)
hei_code_null = allcourses['hei_code'].isnull().sum(axis = 0)
title_null = allcourses['title'].isnull().sum(axis = 0)
code_null = allcourses['code'].isnull().sum(axis = 0)
print(hei_null, hei_code_null, title_null, code_null)

0 0 0 0


All the course in the dataframe are filled in and we can now rearrange dataframe below before joining on our points data.

In [78]:
# Rearranging dataframe
allcourses = allcourses.reset_index()
allcourses = allcourses[['code', 'title', 'hei2', 'hei_code']]
allcourses.columns = ['code', 'title','hei', 'hei_code']
allcourses.set_index('code', inplace=True)
allcourses

,title,hei,hei_code
code,,,
AC120,International Business,American College,AC
AC137,Liberal Arts,American College,AC
AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,AD
AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD
AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,AD
...,...,...,...
WD231,Early Childhood Care and Education,Waterford Institute of Technology,WD
WD232,Business Information Systems,Waterford Institute of Technology,WD
WD139,Civil Engineering,Waterford Institute of Technology,WD


### Joining to the Points and Other Columns
I will first create a datframe with all three years of data, no matter if the course was run over the course of 2019, 2020 and 2021 and give an insight into the points across these years individually. I will will then analyze the points for courses that were in operation across the three years.  

In [79]:
df2021_data = df2021_2[['Course Code', 'CATEGORY (ISCED Description)', 'R1 Points', 'R2 Points ', 'EOS Points','EOS Midpoints', 'Course Level']]
df2021_data.columns = ['code', 'category', 'r1_2021', 'r2_2021', 'EOS_2021', 'MID_2021', 'level']

# Set the index to the code column.
df2021_data.set_index('code', inplace=True)
df2021_data

,category,r1_2021,r2_2021,EOS_2021,MID_2021,level
code,,,,,,
AL605,Engineering and engineering trades,211.0,NaN,211.0,319.0,6
AL630,Health,308.0,NaN,308.0,409.0,6
AL631,Health,311.0,NaN,311.0,400.0,6
AL632,Biological and related sciences,297.0,NaN,297.0,454.0,6
AL650,Business and administration,NaN,NaN,NaN,351.0,6
...,...,...,...,...,...,...
WD211,Information and Communication Technologies (ICTs),270.0,NaN,270.0,392.0,8
WD212,Personal services,262.0,NaN,262.0,304.0,8
WD230,Engineering and engineering trades,230.0,230.0,230.0,361.0,8


In [80]:
df2020_data = df2020[['COURSE CODE2', 'R1 POINTS', 'R2 POINTS', 'EOS','EOS Mid-point']]
df2020_data.columns = ['code', 'r1_2020', 'r2_2020', 'EOS_2020', 'MID_2020']

# Set the index to the code column.
df2020_data.set_index('code', inplace=True)
df2020_data

,r1_2020,r2_2020,EOS_2020,MID_2020
code,,,,
AC120,209.0,NaN,209.0,280.0
AC137,252.0,NaN,252.0,270.0
AD101,NaN,NaN,NaN,NaN
AD102,NaN,NaN,NaN,NaN
AD103,NaN,NaN,NaN,NaN
...,...,...,...,...
WD208,188.0,NaN,188.0,339.0
WD210,279.0,NaN,279.0,337.0
WD211,271.0,NaN,271.0,318.0


In [81]:
df2019_data = df2019[['Course Code', 'EOS', 'Mid']]
df2019_data.columns = ['code', 'EOS_2019', 'MID_2019']

# Set the index to the code column.
df2019_data.set_index('code', inplace=True)
df2019_data

,EOS_2019,MID_2019
code,,
AL801,304.0,328.0
AL802,301.0,306.0
AL803,309.0,337.0
AL805,329.0,442.0
AL810,307.0,349.0
...,...,...
WD188,206.0,339.0
WD205,208.0,441.0
WD206,191.0,322.0


In [82]:
# Validate if this worked
df2019_data.loc['AL763']

EOS_2019    626.0
MID_2019    820.0
Name: AL763, dtype: float64

In [83]:
# Join 2021 points to allcourses.
allcourses2 = allcourses.join(df2021_data)
allcourses2

# Join 2020 points to allcourses.
allcourses3 = allcourses2.join(df2020_data)
allcourses3

# Join 2019 points to allcourses.
allcourses_final = allcourses3.join(df2019_data)
allcourses_final

,title,hei,hei_code,category,r1_2021,r2_2021,EOS_2021,MID_2021,level,r1_2020,r2_2020,EOS_2020,MID_2020,EOS_2019,MID_2019
code,,,,,,,,,,,,,,,
AC120,International Business,American College,AC,Business and administration,294.0,294.0,294.0,352.0,8.0,209.0,NaN,209.0,280.0,234.0,269.0
AC137,Liberal Arts,American College,AC,Humanities (except languages),271.0,270.0,270.0,334.0,8.0,252.0,NaN,252.0,270.0,252.0,275.0
AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,AD,Arts,554.0,NaN,554.0,604.0,8.0,NaN,NaN,NaN,NaN,NaN,550.0
AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD,Arts,538.0,NaN,538.0,625.0,8.0,NaN,NaN,NaN,NaN,NaN,635.0
AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,AD,Arts,505.0,NaN,505.0,581.0,8.0,NaN,NaN,NaN,NaN,NaN,545.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WD231,Early Childhood Care and Education,Waterford Institute of Technology,WD,Welfare,266.0,NaN,266.0,366.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
WD232,Business Information Systems,Waterford Institute of Technology,WD,Business and administration,261.0,261.0,261.0,329.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
WD139,Civil Engineering,Waterford Institute of Technology,WD,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,206.0,379.0,200.0,364.0


In [84]:
allcourses_final = allcourses_final.reset_index()
allcourses_final

,code,title,hei,hei_code,category,r1_2021,r2_2021,EOS_2021,MID_2021,level,r1_2020,r2_2020,EOS_2020,MID_2020,EOS_2019,MID_2019
0,AC120,International Business,American College,AC,Business and administration,294.0,294.0,294.0,352.0,8.0,209.0,NaN,209.0,280.0,234.0,269.0
1,AC137,Liberal Arts,American College,AC,Humanities (except languages),271.0,270.0,270.0,334.0,8.0,252.0,NaN,252.0,270.0,252.0,275.0
2,AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,AD,Arts,554.0,NaN,554.0,604.0,8.0,NaN,NaN,NaN,NaN,NaN,550.0
3,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD,Arts,538.0,NaN,538.0,625.0,8.0,NaN,NaN,NaN,NaN,NaN,635.0
4,AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,AD,Arts,505.0,NaN,505.0,581.0,8.0,NaN,NaN,NaN,NaN,NaN,545.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,WD231,Early Childhood Care and Education,Waterford Institute of Technology,WD,Welfare,266.0,NaN,266.0,366.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
1808,WD232,Business Information Systems,Waterford Institute of Technology,WD,Business and administration,261.0,261.0,261.0,329.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
1809,WD139,Civil Engineering,Waterford Institute of Technology,WD,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,206.0,379.0,200.0,364.0
1810,WD149,Early Childhood Studies,Waterford Institute of Technology,WD,NaN,NaN,NaN,NaN,NaN,NaN,271.0,NaN,271.0,368.0,288.0,336.0


In [85]:
CAO = allcourses_final[['code', 'title', 'hei', 'EOS_2021', 'MID_2021', 'EOS_2020', 'MID_2020', 'EOS_2019', 'MID_2019']]
CAO

,code,title,hei,EOS_2021,MID_2021,EOS_2020,MID_2020,EOS_2019,MID_2019
0,AC120,International Business,American College,294.0,352.0,209.0,280.0,234.0,269.0
1,AC137,Liberal Arts,American College,270.0,334.0,252.0,270.0,252.0,275.0
2,AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,554.0,604.0,NaN,NaN,NaN,550.0
3,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,538.0,625.0,NaN,NaN,NaN,635.0
4,AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,505.0,581.0,NaN,NaN,NaN,545.0
...,...,...,...,...,...,...,...,...,...
1807,WD231,Early Childhood Care and Education,Waterford Institute of Technology,266.0,366.0,NaN,NaN,NaN,NaN
1808,WD232,Business Information Systems,Waterford Institute of Technology,261.0,329.0,NaN,NaN,NaN,NaN
1809,WD139,Civil Engineering,Waterford Institute of Technology,NaN,NaN,206.0,379.0,200.0,364.0
1810,WD149,Early Childhood Studies,Waterford Institute of Technology,NaN,NaN,271.0,368.0,288.0,336.0


I will remove any course from the data set where there are no values at all for the points across the three years as there is no need for them in dataset.

In [86]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
# https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-a-certain-column-is-nan
CAO.dropna(axis = 0, subset = ['EOS_2021', 'MID_2021', 'EOS_2020', 'MID_2020', 'EOS_2019', 'MID_2019'], how = 'all', inplace=True)


<ipython-input-86-e620ddfc32b3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CAO.dropna(axis = 0, subset = ['EOS_2021', 'MID_2021', 'EOS_2020', 'MID_2020', 'EOS_2019', 'MID_2019'], how = 'all', inplace=True)


In [87]:
CAO

,code,title,hei,EOS_2021,MID_2021,EOS_2020,MID_2020,EOS_2019,MID_2019
0,AC120,International Business,American College,294.0,352.0,209.0,280.0,234.0,269.0
1,AC137,Liberal Arts,American College,270.0,334.0,252.0,270.0,252.0,275.0
2,AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,554.0,604.0,NaN,NaN,NaN,550.0
3,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,538.0,625.0,NaN,NaN,NaN,635.0
4,AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,505.0,581.0,NaN,NaN,NaN,545.0
...,...,...,...,...,...,...,...,...,...
1807,WD231,Early Childhood Care and Education,Waterford Institute of Technology,266.0,366.0,NaN,NaN,NaN,NaN
1808,WD232,Business Information Systems,Waterford Institute of Technology,261.0,329.0,NaN,NaN,NaN,NaN
1809,WD139,Civil Engineering,Waterford Institute of Technology,NaN,NaN,206.0,379.0,200.0,364.0
1810,WD149,Early Childhood Studies,Waterford Institute of Technology,NaN,NaN,271.0,368.0,288.0,336.0


In [88]:
CAO.dtypes

code         object
title        object
hei          object
EOS_2021    float64
MID_2021    float64
EOS_2020    float64
MID_2020    float64
EOS_2019    float64
MID_2019    float64
dtype: object

In [89]:
CAO_all3_1 = allcourses.join(df2021_data, how = 'inner')
CAO_all3_1

,title,hei,hei_code,category,r1_2021,r2_2021,EOS_2021,MID_2021,level
code,,,,,,,,,
AC120,International Business,American College,AC,Business and administration,294.0,294.0,294.0,352.0,8
AC137,Liberal Arts,American College,AC,Humanities (except languages),271.0,270.0,270.0,334.0,8
AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,AD,Arts,554.0,NaN,554.0,604.0,8
AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD,Arts,538.0,NaN,538.0,625.0,8
AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,AD,Arts,505.0,NaN,505.0,581.0,8
...,...,...,...,...,...,...,...,...,...
WD211,Creative Computing,Waterford Institute of Technology,WD,Information and Communication Technologies (ICTs),270.0,NaN,270.0,392.0,8
WD212,Recreation and Sport Management,Waterford Institute of Technology,WD,Personal services,262.0,NaN,262.0,304.0,8
WD230,Mechanical and Manufacturing Engineering,Waterford Institute of Technology,WD,Engineering and engineering trades,230.0,230.0,230.0,361.0,8


In [90]:
CAO_all3_2 = CAO_all3_1.join(df2020_data, how = 'inner')
CAO_all3_2

,title,hei,hei_code,category,r1_2021,r2_2021,EOS_2021,MID_2021,level,r1_2020,r2_2020,EOS_2020,MID_2020
code,,,,,,,,,,,,,
AC120,International Business,American College,AC,Business and administration,294.0,294.0,294.0,352.0,8,209.0,NaN,209.0,280.0
AC137,Liberal Arts,American College,AC,Humanities (except languages),271.0,270.0,270.0,334.0,8,252.0,NaN,252.0,270.0
AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,AD,Arts,554.0,NaN,554.0,604.0,8,NaN,NaN,NaN,NaN
AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD,Arts,538.0,NaN,538.0,625.0,8,NaN,NaN,NaN,NaN
AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,AD,Arts,505.0,NaN,505.0,581.0,8,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WD208,Manufacturing Engineering,Waterford Institute of Technology,WD,Engineering and engineering trades,317.0,NaN,317.0,368.0,7,188.0,NaN,188.0,339.0
WD210,Software Systems Development,Waterford Institute of Technology,WD,Information and Communication Technologies (ICTs),260.0,NaN,260.0,373.0,8,279.0,NaN,279.0,337.0
WD211,Creative Computing,Waterford Institute of Technology,WD,Information and Communication Technologies (ICTs),270.0,NaN,270.0,392.0,8,271.0,NaN,271.0,318.0


In [91]:
CAO_all3 = CAO_all3_2.join(df2019_data, how = 'inner')
CAO_all3

,title,hei,hei_code,category,r1_2021,r2_2021,EOS_2021,MID_2021,level,r1_2020,r2_2020,EOS_2020,MID_2020,EOS_2019,MID_2019
code,,,,,,,,,,,,,,,
AC120,International Business,American College,AC,Business and administration,294.0,294.0,294.0,352.0,8,209.0,NaN,209.0,280.0,234.0,269.0
AC137,Liberal Arts,American College,AC,Humanities (except languages),271.0,270.0,270.0,334.0,8,252.0,NaN,252.0,270.0,252.0,275.0
AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,AD,Arts,554.0,NaN,554.0,604.0,8,NaN,NaN,NaN,NaN,NaN,550.0
AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,AD,Arts,538.0,NaN,538.0,625.0,8,NaN,NaN,NaN,NaN,NaN,635.0
AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,AD,Arts,505.0,NaN,505.0,581.0,8,NaN,NaN,NaN,NaN,NaN,545.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WD208,Manufacturing Engineering,Waterford Institute of Technology,WD,Engineering and engineering trades,317.0,NaN,317.0,368.0,7,188.0,NaN,188.0,339.0,195.0,330.0
WD210,Software Systems Development,Waterford Institute of Technology,WD,Information and Communication Technologies (ICTs),260.0,NaN,260.0,373.0,8,279.0,NaN,279.0,337.0,271.0,329.0
WD211,Creative Computing,Waterford Institute of Technology,WD,Information and Communication Technologies (ICTs),270.0,NaN,270.0,392.0,8,271.0,NaN,271.0,318.0,275.0,322.0


In [95]:

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html
CAO_all3_1 = allcourses.join(df2021_data, how = 'inner')
CAO_all3_2 = CAO_all3_1.join(df2020_data, how = 'inner')
CAO_all3 = CAO_all3_2.join(df2019_data, how = 'inner')

CAO_all3 = CAO_all3.reset_index()

CAO_3 = CAO_all3[['code', 'title', 'hei', 'EOS_2021', 'MID_2021', 'EOS_2020', 'MID_2020', 'EOS_2019', 'MID_2019', 'level']]
CAO_3

,code,title,hei,EOS_2021,MID_2021,EOS_2020,MID_2020,EOS_2019,MID_2019,level
0,AC120,International Business,American College,294.0,352.0,209.0,280.0,234.0,269.0,8
1,AC137,Liberal Arts,American College,270.0,334.0,252.0,270.0,252.0,275.0,8
2,AD101,"First Year Art and Design (Common Entry,portfo...",National College of Art and Design,554.0,604.0,NaN,NaN,NaN,550.0,8
3,AD102,Graphic Design and Moving Image Design (portfo...,National College of Art and Design,538.0,625.0,NaN,NaN,NaN,635.0,8
4,AD103,Textile and Surface Design and Jewellery and O...,National College of Art and Design,505.0,581.0,NaN,NaN,NaN,545.0,8
...,...,...,...,...,...,...,...,...,...,...
1066,WD208,Manufacturing Engineering,Waterford Institute of Technology,317.0,368.0,188.0,339.0,195.0,330.0,7
1067,WD210,Software Systems Development,Waterford Institute of Technology,260.0,373.0,279.0,337.0,271.0,329.0,8
1068,WD211,Creative Computing,Waterford Institute of Technology,270.0,392.0,271.0,318.0,275.0,322.0,8
1069,WD212,Recreation and Sport Management,Waterford Institute of Technology,262.0,304.0,270.0,349.0,274.0,311.0,8


In [96]:
CAO_3.dropna(axis = 0, subset = ['EOS_2021', 'MID_2021', 'EOS_2020', 'MID_2020', 'EOS_2019', 'MID_2019'], how = 'any', inplace=True)

<ipython-input-96-035e7e10465e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CAO_3.dropna(axis = 0, subset = ['EOS_2021', 'MID_2021', 'EOS_2020', 'MID_2020', 'EOS_2019', 'MID_2019'], how = 'any', inplace=True)


In [97]:
CAO_3

,code,title,hei,EOS_2021,MID_2021,EOS_2020,MID_2020,EOS_2019,MID_2019,level
0,AC120,International Business,American College,294.0,352.0,209.0,280.0,234.0,269.0,8
1,AC137,Liberal Arts,American College,270.0,334.0,252.0,270.0,252.0,275.0,8
9,AD215,Visual Culture,National College of Art and Design,300.0,380.0,320.0,389.0,300.0,338.0,8
12,AL630,Pharmacy Technician,Athlone Institute of Technology,308.0,409.0,295.0,379.0,306.0,366.0,6
13,AL631,Dental Nursing,Athlone Institute of Technology,311.0,400.0,298.0,388.0,326.0,379.0,6
...,...,...,...,...,...,...,...,...,...,...
1066,WD208,Manufacturing Engineering,Waterford Institute of Technology,317.0,368.0,188.0,339.0,195.0,330.0,7
1067,WD210,Software Systems Development,Waterford Institute of Technology,260.0,373.0,279.0,337.0,271.0,329.0,8
1068,WD211,Creative Computing,Waterford Institute of Technology,270.0,392.0,271.0,318.0,275.0,322.0,8
1069,WD212,Recreation and Sport Management,Waterford Institute of Technology,262.0,304.0,270.0,349.0,274.0,311.0,8


---

## References 